In [9]:
!pip install spacy
!pip install pyLDAvis

     |████████████████████████████████| 14.2MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 215kB 19.0MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 25.1MB/s eta 0:00:01
     |████████████████████████████████| 276kB 50.3MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 50.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 24.5MB/s eta 0:00:01
  Found existing installation: importlib-metadata 0.17
    Uninstalling importlib-metadata-0.17:
      Successfully uninstalled importlib-metadata-0.17


In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from tqdm import tqdm

In [78]:
comments=pd.read_csv('./removed_january.csv')

In [79]:
comments.shape

(347142, 6)

In [80]:
comments.isnull().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      0
comments         183
dtype: int64

In [81]:
comments.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...
3,2515,1107,2009-03-27,9193,Holly,Stephanie's offered all the most important thi...
4,2515,2175,2009-05-09,7048,Alessandra,"Stephanie was really nice, ftiendly and helpfu..."


In [82]:
comments.dropna(inplace=True)

In [83]:
comments.isnull().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [84]:
import re
import numpy as np

In [85]:
import gensim

In [86]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [87]:
import spacy

In [88]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [89]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ikhyvicky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [91]:
# Convert email body to list
data = comments.comments.values.tolist()

In [0]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [0]:
data_words=list(sent_to_words(data))

In [0]:
print(data_words[3])

['stephanie', 'offered', 'all', 'the', 'most', 'important', 'things', 'warm', 'welcome', 'into', 'comfortable', 'home', 'comfortable', 'bed', 'in', 'quiet', 'room', 'fresh', 'clean', 'towels', 'blankets', 'and', 'easy', 'access', 'to', 'manhattan', 'finding', 'myself', 'travelling', 'to', 'nyc', 'in', 'the', 'future', 'feel', 'already', 'have', 'open', 'invitation', 'to', 'make', 'home', 'away', 'from', 'home', 'through', 'stephanie', 'generousity']


In [14]:
from gensim.models.phrases import Phrases, Phraser

In [0]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [0]:
print(trigram_mod[bigram_mod[data_words[35]]])

['best', 'host', 'ever', 'had', 'stefanie', 'and', 'her', 'family', 'are', 'very', 'kind', 'helpful', 'and', 'accommodating', 'you', 'will', 'feel', 'at', 'home', 'here', 'immediately', 'the', 'place', 'is', 'very', 'nice', 'clean', 'and', 'you', 'can', 'beat', 'the', 'access', 'to', 'both', 'the', 'and', 'as', 'well', 'as', 'the', 'and', 'subway', 'lines', 'both', 'each', 'blocks', 'away', 'will', 'definitely', 'come', 'back']


In [0]:
#remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
data_words_nostops = remove_stopwords(data_words)

In [0]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [0]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

In [0]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [15]:
import pickle

In [0]:
f=open("/content/gdrive/My Drive/Capstone stuff/january stuff/data_lemmatized.pkl",'wb')
pickle.dump(data_lemmatized,f)
f.close()

In [16]:
data_lemmatized = []
with (open("data_lemmatized.pkl", "rb")) as openfile:
    while True:
        try:
            data_lemmatized.append(pickle.load(openfile))
        except EOFError:
            break

In [17]:
data_lemmatized=data_lemmatized[0]

In [18]:
len(data_lemmatized)

346959

In [19]:
#create a dictionary and corpus for (LDA topic modeling)

#creating the dictionary

idtoword=corpora.Dictionary(data_lemmatized)
#create corpus

texts=data_lemmatized

corpus=[idtoword.doc2bow(text) for text in texts]

In [45]:
len(corpus)

346959

# Gensim LDA Model -Topic Modeling

In [26]:
#build our LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=idtoword,
                                           num_topics=20,
                                           workers=3,
                                           random_state=100,
                                           eval_every=1,
                                           chunksize=100,
                                           passes=10)


In [72]:
f=open("./model/lda.pkl",'wb')
pickle.dump(lda_model,f)
f.close()

In [73]:
g=open("./model/idtoword.pkl",'wb')
pickle.dump(idtoword,g)
g.close()

In [74]:
h=open("./model/corpus.pkl",'wb')
pickle.dump(corpus,h)
h.close()

In [68]:
#pickle.dump(lda_model, open("./model/lda.pickle", "wb"))
#pickle.dump(idtoword, open("./model/idatoword.pickle", "wb"))
#pickle.dump(corpus, open("./model/corpus.pickle", "wb"))


In [27]:
print(lda_model.print_topics())

[(0, '0.036*"subway" + 0.034*"great" + 0.033*"walk" + 0.029*"place" + 0.028*"apartment" + 0.026*"close" + 0.023*"restaurant" + 0.022*"location" + 0.016*"manhattan" + 0.015*"easy"'), (1, '0.065*"car" + 0.065*"fit" + 0.049*"logement" + 0.049*"limpio" + 0.048*"confortable" + 0.038*"service" + 0.030*"mas" + 0.029*"ensure" + 0.021*"com" + 0.019*"si"'), (2, '0.071*"room" + 0.041*"bed" + 0.034*"bathroom" + 0.031*"clean" + 0.027*"bedroom" + 0.025*"kitchen" + 0.024*"space" + 0.020*"good" + 0.018*"comfortable" + 0.018*"nice"'), (3, '0.077*"value" + 0.076*"money" + 0.068*"worth" + 0.030*"necessary" + 0.029*"chill" + 0.026*"deli" + 0.023*"appliance" + 0.018*"bedding" + 0.017*"resolve" + 0.014*"improve"'), (4, '0.089*"harlem" + 0.051*"del" + 0.048*"hay" + 0.043*"amable" + 0.035*"recomendable" + 0.033*"era" + 0.033*"solo" + 0.028*"linda" + 0.027*"cama" + 0.023*"alle"'), (5, '0.054*"stay" + 0.040*"new" + 0.035*"home" + 0.035*"place" + 0.028*"make" + 0.027*"york" + 0.025*"feel" + 0.025*"back" + 0.023*

In [30]:
lda_model.save('lda.model')

In [32]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, idtoword)
pyLDAvis.display(vis)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [33]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))



Perplexity:  -10.364497799337316


In [35]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=idtoword, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy has detected that you (may be) writing to an array with
overlapping memory from np.broadcast_arrays. If this is intentional
set the WRITEABLE flag True or make a copy immediately before writing.
  i, j, x)
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy ha


Coherence Score:  0.571454929572421


In [92]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [93]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

In [94]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [116]:
df_dominant_topic.shape

(346959, 5)

In [95]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,12.0,0.6099,"great, location, place, stay, host, clean, wou...",Stephanie was a wonderful host! Her apartment ...
1,1,8.0,0.4299,"tre, appartement, est, pour, dan, bien, metro,...",Such a wonderful place and very close to the m...
2,2,5.0,0.2337,"stay, new, home, place, make, york, feel, back...",I just got back from a trip to NYC during whic...
3,3,5.0,0.3530,"stay, new, home, place, make, york, feel, back...",Stephanie's offered all the most important thi...
4,4,7.0,0.4788,"place, nice, good, really, host, clean, stay, ...","Stephanie was really nice, ftiendly and helpfu..."
5,5,7.0,0.6224,"place, nice, good, really, host, clean, stay, ...",I was pleasantly surprised with my whole stay ...
6,6,0.0,0.5985,"subway, great, walk, place, apartment, close, ...",Stephanie's apt was great. Very convenient to ...
7,7,5.0,0.3146,"stay, new, home, place, make, york, feel, back...",my husband and i had such a great time staying...
8,8,0.0,0.2981,"subway, great, walk, place, apartment, close, ...","When I first arrived at Stephanie's, she was v..."
9,9,5.0,0.6833,"stay, new, home, place, make, york, feel, back...",awesome couldn't have been better.


In [97]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [98]:
sia = SentimentIntensityAnalyzer()

In [104]:
example=df_dominant_topic.loc[:10,:]

In [105]:
example.shape

(11, 5)

In [115]:
for sentence in example.Text:
    # Assess sentiment.
    sentiment = sia.polarity_scores(sentence)["compound"]
    print(sentence)
    print(sentiment)
    print('\n')

Stephanie was a wonderful host! Her apartment was beautiful and extremely clean. We had a very nice (and affordable!) stay in the city with private room and own bathroom.
0.9357


Such a wonderful place and very close to the metro station on the line 1 2 3. She was awesome with us, it was a pleasure to stay in her appt. she's french, so last word ...
Merci encore stephanie pour ton hospitalité. On a passé un super séjour avec toi. Très disponible pour nous avec pleins de conseils, je vous recommande particulierement cette adresse qui plus est pour un prix raisonnable. Le metro est juste a 1 bloc de l'appartement et la ligne 1-2-3 est la plus centrale et la plus pratique de NYC. son quartier est vraiment tres sympa en plus !!! Allez y les yeux fermés.
0.9537


I just got back from a trip to NYC during which I stayed at Stephanie's appartment. The bedroom and private bathroom that comes with it were both kept very clean and are spacious so that I could unpack my stuff and put my clothes 

In [119]:
sentiment_scores=[]
for sentence in df_dominant_topic.Text:
    sentiment = sia.polarity_scores(sentence)["compound"]
    sentiment_scores.append(sentiment)
    

In [129]:
df_dominant_topic.drop('e',axis=1,inplace=True)

In [125]:
df_dominant_topic['sentiment_score'] = pd.Series(sentiment_scores, index=df_dominant_topic.index)

In [137]:
keyword_scores=[]
for sentence in df_dominant_topic.Keywords:
    sentiment = sia.polarity_scores(sentence)["compound"]
    keyword_scores.append(sentiment)

In [130]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,12.0,0.6099,"great, location, place, stay, host, clean, wou...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,8.0,0.4299,"tre, appartement, est, pour, dan, bien, metro,...",Such a wonderful place and very close to the m...,0.9537
2,2,5.0,0.2337,"stay, new, home, place, make, york, feel, back...",I just got back from a trip to NYC during whic...,0.9620
3,3,5.0,0.3530,"stay, new, home, place, make, york, feel, back...",Stephanie's offered all the most important thi...,0.9626
4,4,7.0,0.4788,"place, nice, good, really, host, clean, stay, ...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [156]:
negative_sentiments=df_dominant_topic[df_dominant_topic['sentiment_score']<0]

In [161]:
negative_sentiments.reset_index(drop=True,inplace=True)

In [162]:
negative_sentiments.shape

(15998, 6)

In [172]:
negative_sentiments.sort_values('sentiment_score').head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
8982,186980,11.0,0.9185,"und, ist, die, man, war, der, super, be, sehr,...",*** Kommunikation *** Die Kommunikation mit Da...,-0.9992
7343,152568,11.0,0.8962,"und, ist, die, man, war, der, super, be, sehr,...",Die Unterkunft war toll.\nCheck-in war unprobl...,-0.9988
14738,313770,11.0,0.9731,"und, ist, die, man, war, der, super, be, sehr,...",Wir haben 3 Nächte in der Unterkunft verbracht...,-0.9987
2250,49840,11.0,0.9769,"und, ist, die, man, war, der, super, be, sehr,...",Ich habe gemischte Gefühle was die Unterkunft ...,-0.9986
5048,106495,11.0,0.9824,"und, ist, die, man, war, der, super, be, sehr,...","Zuerst möchte ich sagen, dass wir Wei nicht pe...",-0.9985
6616,137464,11.0,0.9700,"und, ist, die, man, war, der, super, be, sehr,...",Insgesamt betrachtet war die Wohnung für das G...,-0.9978
10692,223276,11.0,0.9412,"und, ist, die, man, war, der, super, be, sehr,...","Check in problemlos, Lage der Wohnung war okay...",-0.9976
1363,30507,11.0,0.9304,"und, ist, die, man, war, der, super, be, sehr,...",Bei der Ankunft lief alles wie vorher mit Clau...,-0.9975
2832,61234,11.0,0.9104,"und, ist, die, man, war, der, super, be, sehr,...",Koseki hat auf unsere Anfrage und E-Mails imme...,-0.9975
8885,184906,11.0,0.8735,"und, ist, die, man, war, der, super, be, sehr,...",Unser Aufenthalt in New York war in diesem App...,-0.9974


In [177]:
negative_sentiments.Text.iloc[8982]

'*** Kommunikation *** Die Kommunikation mit Dan war großartig! Er hat immer sehr schnell geantwortet und war immer sehr bemüht uns unseren Aufenthalt so toll wie möglich zu gestalten. Wir haben ihn am letzten Tag persönlich getroffen und er wirkte sehr freundlich und zuvorkommend. *** Die Lage *** Wir waren für 6 Nächte in Dans Wohnung im East Village. Uns hat die Lage der Unterkunft ausgesprochen gut gefallen, da man mitten im Geschehen war und von dort sehr gut mit dem Bus (M14a) oder der U-Bahn (F oder L) überall schnell hin kam. Man war so zentral, dass man sogar kurz zurück konnte, um z.B. Einkäufe abzulegen. Falls man in der näheren Umgebung der Unterkunft essen möchte, dann gibt es unzählige gute Restaurants, Bars oder Cafes. Insbesondere am Wochenende (abends) ist es sehr lebhaft, aber dennoch nicht besonders laut in der Unterkunft, da diese zum Hinterhof hinaus geht und der Straßenlärm nicht zu hören ist. Die Lage ist also top! *** Die Wohnung *** Den Schlüssel für die Wohnun

In [143]:
negative_sentiments.Dominant_Topic.nunique() #this just means people  can talk negative things for each topic

20

WHAT I REALISED
<br>the topic classification via LDA although not the best poppped up few interesting things:
1. Non english words popped up! and they form a classification on thier own
2. Subsequently affected sentiment analysis as the german words turned up to be the most negative, which they are not 
3. Need to translate and remodel to get a better understanding

### Hypertuning PART

In [75]:
#hypertuning the model

def compute_coherence_values(corpus, dictionary, k, a):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           per_word_topics=True,
                                           workers=3)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=idtoword, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
#iterate the best number of topics and at what alpha

In [76]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 3
max_topics = 8
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
#beta = list(np.arange(0.01, 1, 0.3))
#beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
#corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
#               corpus]
#corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    #for i in range(len(corpus_sets)):
        # iterate through number of topics
    for k in topics_range:
            # iterate through alpha values
        for a in alpha:
                # iterare through beta values
            #for b in beta:
                    # get the coherence score for the given parameters
            cv = compute_coherence_values(corpus=corpus, dictionary=idtoword, 
                                                  k=k, a=a)
                    # Save the model results
                #model_results['Validation_Set'].append(corpus_title[i])
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            #model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
                    
            pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()














  0%|          | 0/540 [00:00<?, ?it/s]

KeyboardInterrupt: 

Process ForkPoolWorker-146:
Traceback (most recent call last):
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/numpy/random/_pickle.py", line 62, in __randomstate_ctor
    def __randomstate_ctor(bit_generator_name='MT19937'):
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recen